In [1]:
import numpy as np
import pandas as pd
from background.data_load import data_load
from background.data_split import train_test_split, train_test_mini_batch, df_shuffle
from background.sequential import Sequential, Input, Dense
from background.evaluate import evaluate_epoch


In [3]:
df = data_load()
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [7]:
df = df_shuffle(df)
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,23.625000,29.948654,5.688038,35.987172,146.568562,82.394624,-0.274902,-1.121848,1
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,137.242188,46.454740,0.045257,-0.438858,59.495819,77.755357,0.719748,-1.183162,0
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [15]:
X_train, y_train, X_test, y_test = train_test_split(df)
X_train.head()

c:\AI_bootcamp_python\CP1\cp1_project\py_file\background\data_split.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = (X[feature] - x_min[feature]) / (x_max[feature] - x_min[feature])


,x1,x2,x3,x4,x5,x6,x7,x8
4,0.624562,0.288209,0.199580,0.108995,0.036281,0.267176,0.321601,0.087668
5,0.680052,0.940915,0.101802,0.010892,0.021278,0.173681,0.415569,0.156854
6,0.000000,0.051964,0.985669,0.962179,1.000000,1.000000,0.000000,0.000180
7,0.864815,0.987598,0.027229,0.004138,0.009153,0.083678,0.530236,0.270490
8,0.666644,0.522882,0.262437,0.121321,0.183648,0.712134,0.142798,0.014217


In [4]:
model = Sequential([Input(8), Dense(8), Dense(1, activation='sigmoid')])

In [10]:
def predict(X_batch, y_batch, model):
    model.input = X_batch
    y_pred_prob = model.output()
    return y_pred_prob

In [11]:
def classification(x):
        if x < 0.5:
            return 0
        else:
            return 1
        # return 0 if x < 0.5 else 1
        # return round(x, 0)

classify = np.vectorize(classification)


In [ ]:
df = data_load()

# train, test 데이터로 나누기
X_train, y_train, X_test, y_test = train_test_split(df)
# 미니배치(하나에 4개 데이터)로 나누기
X_batch_list, y_batch_list, test_X_batch_list, test_y_batch_list = train_test_mini_batch(X_train, y_train, X_test, y_test)


# 모델 실행 및 정확도, 손실함수 측정
print("Train data")
evaluate_epoch(X_batch_list, y_batch_list, model, 1)    # epoch 1
print("Test data")
evaluate_epoch(test_X_batch_list, test_y_batch_list, model, 1)
print(model.tail.W)

c:\AI_bootcamp_python\CP1\cp1_project\py_file\background\data_split.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = (X[feature] - x_min[feature]) / (x_max[feature] - x_min[feature])


Train data
[Mini-Batch 1] TrainData - Loss = 0.694, Accuracy = 0.5


ValueError: non-broadcastable output operand with shape (8,) doesn't match the broadcast shape (4,8)

In [14]:
X_batch = X_batch_list[0]
y_batch = y_batch_list[0]

In [15]:
y_pred_prob = predict(X_batch, y_batch, model)
y_pred = classify(y_pred_prob)

In [20]:
# def back_prop(model, y_pred_prob, y_batch, learning_rate=1):    # (체크) 미니배치 단위로 손실함수 평균 내서 역전파 하기
learning_rate = 1

def sigmoid_prime(y):
    return y * (1 - y)
# input(X), output(predict), y, error(cross_entropy)
def cross_entropy_prime(y_pred_prob, y_batch):  # binary cross entropy
    return y_pred_prob - y_batch

iterator = model.tail
while iterator != model.head:
    # dError/dY
    if iterator == model.tail:
        error = cross_entropy_prime(y_pred_prob, y_batch)
    # else:
    #     error = np.dot(delta, iterator.prev.W.T)    # 요기
    # dY/dy
    if iterator.activation == 'sigmoid':
        delta = error * sigmoid_prime(y_pred_prob)
    elif iterator.activation == 'linear':
        delta = error
    # dy/dw
    print(iterator.W)
    delta_mean = np.mean(delta)
    iterator.b -= learning_rate * delta_mean
    iterator.W -= learning_rate * np.dot(iterator.X.T, delta)
    print(iterator.W)
    iterator = iterator.prev

[[ 0.03464753]
 [ 0.01853462]
 [-0.00427367]
 [ 0.01727433]
 [ 0.00210227]
 [-0.02888851]
 [ 0.01284165]
 [-0.06023316]]
[[ 0.03519927]
 [ 0.01827295]
 [-0.00352245]
 [ 0.01617977]
 [ 0.00149418]
 [-0.02914895]
 [ 0.01351639]
 [-0.06183886]]
[[-0.19825432 -0.20364418 -0.20533275 -0.2085526  -0.21149877 -0.21306173
  -0.21616395 -0.18829994]
 [-0.00460489 -0.00333842 -0.0044082  -0.00914708 -0.00641973  0.00244711
  -0.00657613 -0.00456942]
 [ 0.2385426   0.23700083  0.24045434  0.2266355   0.23341741  0.23228442
   0.23001307  0.23438983]
 [ 0.10170651  0.11288547  0.11043952  0.10547909  0.09916141  0.09635804
   0.10728734  0.10276054]
 [ 0.35833845  0.34115413  0.34639056  0.34454099  0.35057425  0.35414118
   0.34029077  0.33718406]
 [ 0.67052149  0.66618552  0.67209458  0.65207136  0.64825561  0.66095894
   0.66985226  0.65878806]
 [-0.29929485 -0.31227072 -0.3106734  -0.31008015 -0.32644434 -0.30411371
  -0.3090677  -0.30315924]
 [-0.18248996 -0.17128908 -0.17461171 -0.18789489 -

In [ ]:
model = Sequential([Input(8), Dense(1, activation='sigmoid')])
def test(model):
    iterator = model
    iterator.tail.W += np.ones(shape=(8, 1))
print(model.tail.W)
test(model)
print(model.tail.W)

[[ 0.00057836]
 [ 0.01596949]
 [-0.00947748]
 [-0.01421698]
 [ 0.02525306]
 [ 0.01734899]
 [-0.02285538]
 [-0.00879917]]
[[1.00057836]
 [1.01596949]
 [0.99052252]
 [0.98578302]
 [1.02525306]
 [1.01734899]
 [0.97714462]
 [0.99120083]]


In [ ]:
import numpy as np

def back_prop(model, y_pred_prob, y_batch, learning_rate=1):
    def sigmoid_prime(y):
        return y * (1 - y)
    # input(X), output(predict), y, error(cross_entropy)
    def cross_entropy_prime(y_pred_prob, y_batch):  # binary cross entropy
        return y_pred_prob - y_batch

    iterator = model.tail
    while iterator.prev != model.head:
        # dError/dY
        if iterator == model.tail:
            error = cross_entropy_prime(y_pred_prob, y_batch)
        # else:
        #     error = np.dot(delta, iterator.prev.W.T)
        # dY/dy
        if iterator.activation == 'sigmoid':
            delta = error * sigmoid_prime(y_pred_prob)
        elif iterator.activation == 'linear':
            delta = error
        # dy/dw
        iterator.prev.W -= learning_rate * np.dot(iterator.prev.X.T, delta)
        iterator = iterator.prev
back_prop(model, )